# Yelp Open Dataset Visualization

The aim of the project is to visualize the data contained in the [Yelp open dataset](https://www.yelp.com/dataset). The dataset was converted from JSON format into a SQLite database for efficient querying of the 8.4GB dataset. The output from the SQL queries were brought into python for visualization.

The final size of the SQL database is 6.3GB, consisting of 192,609 rows in the business table, 1,637,138 rows in the users table and 6,685,900 rows in the review table.


# SQLite Database Creation

The dataset provided by yelp was split into six JSON files:


| File          | Description                           | File Size (kb) |
|---------------|---------------------------------------|----------------|
| business.json | Business location data and attributes | 138,279,749    |
| checkin.json  | User 'Checkins' to businesses         | 408,807,658    |
| photo.json    | Photo data for businesses             | 25,661,152     |
| review.json   | User reviews of businesses            | 5,347,475,638  |
| tip.json      | User tips of businesses               | 244,535,478    |
| user.json     | User information                      | 2,485,747,393  |

Due to the large file sizes, converting the data from JSON format into a SQL database will allow us to query the data without worrying about memory limitations. We will process only the business, review and user files for the purposes of this project.

The documentation for the JSON files can be found [here](https://www.yelp.com/dataset/documentation/main). Exploring the 'business.json' documentation and data reveals three attributes - "attributes", "categories" and "hours" - that have value branches more than one layer deep. Simply flattening these values as a single entry will lead to multiple values in a field, violating the rules of database normalization. Thus, for these attributes separate category tables will be created to list out each possible value and preventing multiple entries. The new tables created are:

"parking" - parking options available at the business  

"meal" - meal options serviced by the restaurant e.g. lunch, dinner, dessert  

"hours" - opening and closing hours for all seven days of the week  

"ambience" - the ambience of the business e.g. romantic, casual, formal  

"category" - category of the business e.g. restaurants, French, coffee  

"misc" - all other attributes that do are not covered in other tables  


business.json contains a 'business_id' field that is unique to each entry, thus we will use this as the primary key for the business table. A similar id field can be found for the review.json and user.json files. Thus the final schema is shown below.

<img src="SQL Schema Yelp.svg">







# Queries

The following queries were used to generate the CSV files used in the [data visualization](https://github.com/tcwou/Yelp-Data-Exploration/blob/master/Data%20Visualization.ipynb) and [sentiment analysis](https://github.com/tcwou/Yelp-Data-Exploration/blob/master/Sentiment%20Analysis.ipynb) portions of this project, the first five rows of each query is printed.

In [8]:
import pandas as pd

# City, state, latitude, longitude, no of businesses, no of reviews, cities ordered by most restaurants/eateries
""" SELECT city, state, latitude, longitude, COUNT(business_id), AVG(stars) 
        FROM
        (SELECT city, state, b.business_id, stars
        FROM business b
        JOIN category c ON c.business_id = b.business_id
        WHERE c.food = 1
        GROUP BY b.business_id) 
        GROUP BY city ORDER BY COUNT(business_id) DESC"""

print(pd.read_csv('topcities.csv', header=None).head())

           0   1          2           3      4         5
0    Toronto  ON  43.681084  -79.392410  10071  3.465892
1  Las Vegas  NV  36.128933 -115.192815   8266  3.504658
2    Phoenix  AZ  33.518598 -112.063534   5103  3.484617
3   Montréal  QC  45.512597  -73.587750   4508  3.734028
4    Calgary  AB  51.040238 -114.069034   3674  3.509662


In [10]:
# Number of users, Average no of review, rating, review length; grouped by number of year of elite
""" SELECT elite, COUNT(*), AVG(review_count), AVG(average_stars), AVG(LENGTH(text))
        FROM review JOIN user ON user.user_id = review.user_id GROUP BY elite """

print(pd.read_csv('elite.csv', header=None).head())

   0        1           2         3           4
0  0  5338376   41.756468  3.707804  536.649918
1  1   327058  171.620813  3.878414  761.444514
2  2   279978  251.512180  3.851132  816.692208
3  3   193766  301.043491  3.821575  863.226010
4  4   135946  437.617650  3.792993  908.563290


In [6]:
# Get yearly growth of users and reviews

""" 
WITH 
year_new_users as (
SELECT strftime('%Y', yelping_since) year, COUNT(*) new_users
FROM user GROUP BY 1
),
prev_year_new_users as (
SELECT *,
lag(new_users) OVER (ORDER BY year) AS prev_new_users
FROM year_new_users
),
year_new_reviews as (
SELECT strftime('%Y', date) year, COUNT(*) new_reviews
FROM review GROUP BY 1
),
prev_year_new_reviews as (
SELECT *,
lag(new_reviews) OVER (ORDER BY year) AS prev_new_reviews
FROM year_new_reviews
)

SELECT r.year, new_users, new_reviews,
round(100*(new_users-prev_new_users)/prev_new_users,1) as user_growth,
round(100*(new_reviews-prev_new_reviews)/prev_new_reviews,1) as review_growth
FROM prev_year_new_users u
JOIN prev_year_new_reviews r ON r.year = u.year
ORDER BY 1
"""
print(pd.read_csv('year_growth.csv', header=None, index_col=None))

       0       1        2       3       4
0   2004      81       13     NaN     NaN
1   2005    1001      876  1135.0  6638.0
2   2006    5836     5081   483.0   480.0
3   2007   16480    21389   182.0   320.0
4   2008   32544    57347    97.0   168.0
5   2009   63977   101173    96.0    76.0
6   2010  106840   187387    66.0    85.0
7   2011  168467   302867    57.0    61.0
8   2012  182900   367090     8.0    21.0
9   2013  197220   491294     7.0    33.0
10  2014  225437   704862    14.0    43.0
11  2015  238660   952400     5.0    35.0
12  2016  199148  1098786   -16.0    15.0
13  2017  120531  1217673   -39.0    10.0
14  2018   78016  1177662   -35.0    -3.0


In [5]:
# Get 35,000 random restaurant reviews

""" 
SELECT b_id, b_stars, is_open, cats, text, r.useful, r.funny, r.cool, r.stars, u.user_id, u.elite
FROM (SELECT b.business_id b_id, AVG(stars) b_stars, is_open, group_concat(category, ", ") cats
FROM business b
JOIN category c ON b.business_id = c.business_id
WHERE c.food = 1 AND b_id IN (SELECT business_id FROM business ORDER BY RANDOM() LIMIT 35000)
GROUP BY b.business_id)
JOIN review r ON r.business_id = b_id
JOIN user u ON u.user_id = r.user_id
LIMIT 35000
"""

print(pd.read_csv('35k_reviews.csv', header=None).head())

                       0    1   2   \
0  b2jN2mm9Wf3RcrZCgfo1cg  4.0   0   
1  FxLfqxdYPA6Z85PFKaqLrg  2.0   0   
2  AakkkTuGZA2KBodKi2_u8A  3.5   1   
3  d_L-rfS1vT3JMzgCUGtiow  4.0   1   
4  3JxKzWquEbPC3yPIfoCiLw  4.0   1   

                                                  3   \
0  Breweries, Event Planning & Services, Pubs, Di...   
1                        Italian, Salad, Gluten-Free   
2                           Asian Fusion, Vietnamese   
3    Breakfast & Brunch, Tapas/Small Plates, Mexican   
4                                       Asian Fusion   

                                                  4   5   6   7   8   \
0  I was really looking forward to visiting after...   1   0   0   2   
1  Wow. So surprised at the one and two star revi...   0   0   0   4   
2  I cannot believe how things have changed in 3 ...   1   1   0   1   
3  Pick any meat on the planet and the chef will ...   0   0   0   5   
4  Best chinese resto. Highly recommended. 5 star...   0   0   0   5   

 